In [54]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.ensemble import RandomForestRegressor

In [56]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
%matplotlib inline
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

import time
start = time.time()

meta_file = pd.read_csv("./AutoKaggle - Metadata.csv", encoding='cp1252',error_bad_lines=False)
arrOfRows = [64,360,239,316,515,518, 523, 451]
nlp_rows = [239]
tabular_rows = [64,360,316,515,518]
row = 451
competition_name = meta_file['name'].loc[row]
print(meta_file['name'].loc[row])
dir_name = "./" + competition_name 
train_file = dir_name + "/data/train.csv"
row_file = dir_name + "/submission/row.csv"

meta = pd.read_csv(row_file)
def preprocessing(row):
   
    train = pd.read_csv(train_file)
    check_pred = False
    train_X = ''
    train_Y = ''
    test = None
    auxiliary = []
    if meta['name'].loc[0] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv")
        check_pred = False
    elif meta['name'].loc[0] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
      

    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv", encoding='cp1252')
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    else:
        target_name = str(meta['targetName'].loc[0])
        train_Y = train[[target_name]]
        train = train.drop(columns=target_name)
        if type(meta['auxiliaryDataURL'].loc[0]) is str:
            auxiliary_calls = eval(meta['auxiliaryDataURL'].loc[0])
            for call in auxiliary_calls:
                auxi = call
                auxiliary.append(pd.read_csv("./" + meta['name'].loc[0] + "/auxiliary_data/" + auxi + ".csv"))
        if type(meta['auxiliary function calls'].loc[0]) is str:
            auxiliary_functions = eval(meta['auxiliary function calls'].loc[0])
            for call in auxiliary_functions:
                exec(call)
        if type(meta['test set'].loc[0]) is str:
            row_file = dir_name + "/data/test.csv"
            test = pd.read_csv(row_file, encoding='cp1252', error_bad_lines=False)
        if type(meta['preprocessing function call'].loc[0]) is str:
            preprocessing_calls = eval(meta['preprocessing function call'].loc[0])
            for call in preprocessing_calls:
                exec(call)
        train = train.dropna()
        if type(meta["unwanted column"].loc[0]) is str:  # check if there's unwanted column
            column_list = eval(meta["unwanted column"].loc[0])
            train = train.drop(column_list,axis=1)
        if type(meta["numeric column"].loc[0]) is str:
            numeric=eval(meta["numeric column"].loc[0])

        if type(meta['augmentation function calls'].loc[0]) is str:
            augmentation_calls = eval(meta['augmentation function calls'].loc[0])
            for call in augmentation_calls:
                exec(call)
                
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_pred:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))

#         train_X,test_X,train_Y,test_Y = train_test_split(train, train_Y, test_size=0.2)
        X_train, X_val, Y_train, Y_val = train_test_split(train, train_Y, test_size=0.2)
        model = Sequential()
        if type(meta['neural network initialization'].loc[0]) is str:
            neural_net_calls = eval(meta['neural network initialization'].loc[0])
            for call in neural_net_calls:
                exec(call)
        if check_pred:
            return train_X, train_Y, test_X, test_Y, pred_X
        else:
            return train_X, train_Y, test_X, test_Y,None#         if check_pred:


preprocessing(row)

digit-recognizer


ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (33600, 784))

In [ ]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test,X_pred):
    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    else:
        if type(meta["featureExtractor function call"].loc[0]) is not str:
            print('not func')
            return X_train,X_test,X_pred
        extraction_function_calls = str(row_data["function call feature extraction"].loc[0])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(row_data["function parameters feature extraction"].loc[0])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [ ]:
def feature_selection():
    pass

In [ ]:
def estimation(row,X_train,X_test,Y_train, Y_test):
    if row in nlp_rows:
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    else:
        estimation_function_calls = eval(meta["estimator1 function call"].loc[0])
        print(estimation_function_calls)
        if len(estimation_function_calls) == 1:
            if type(meta['neural network initialization'].loc[0]) is str:
                neural_net_calls = eval(meta['neural network initialization'].loc[0])
                for call in neural_net_calls:
                    exec(call)
            else:
                str_call = estimation_function_calls[0]
                str_call = 'estimator' + '=' + str_call
                exec(str_call,globals(),globals())
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                if meta["taskType"].loc[0] == 'classification':
                    print(accuracy_score(Y_test,Y_pred))
                if meta["taskType"].loc[0] == 'regression':
                    print("Mean Squared Error is: ", mean_squared_error(Y_test,Y_pred))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)



In [ ]:
if row in nlp_rows:
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)
if row in tabular_rows:
    X_train,Y_train,X_test,Y_test,X_pred = preprocessing(row)
    X_train,X_test,X_pred = feature_extraction(row,X_train,X_test,X_pred)
    estimation(row,X_train,X_test,Y_train, Y_test)

In [ ]:
# if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
#     train_set = preprocessing(row)
#     X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
#     Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
#     print(Y_pred)

In [ ]:
def postprocessing(estimators,stack):
    pass

In [ ]:
end = time.time()
print("Use", end - start, "seconds to run this.")

In [ ]:
meta

In [ ]:
meta['test set'].loc[0]

In [ ]:
A = preprocessing(row)

In [ ]:
train_X, train_Y, test_X, test_Y, pred = preprocessing(row)

In [33]:
train_Y

,Sales
8529,5482
214484,3180
160970,6859
382268,9089
505530,9029
654252,9640
248423,9473
955511,5107
701507,5154
799746,6676


In [34]:
estimation(row,train_X, test_X, train_Y, test_Y)

['RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)']


/Users/Osborn/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


('Mean Squared Error is: ', 3585169.497040621)


In [35]:
end = time.time()
print("Use", end - start, "seconds to run this.")

('Use', 53.83630609512329, 'seconds to run this.')


In [36]:
!conda list

# packages in environment at /Users/Osborn/anaconda3/envs/myTensorflow:
#
# Name                    Version                   Build  Channel
absl-py                   0.7.1                    pypi_0    pypi
appnope                   0.1.0            py35hd172556_0  
astor                     0.7.1                    pypi_0    pypi
attrs                     19.1.0                     py_0  
automat                   0.7.0                    py35_0  
backcall                  0.1.0                    py35_0  
bleach                    2.1.4                    py35_0  
ca-certificates           2019.1.23                     0    anaconda
certifi                   2018.8.24                py35_1    anaconda
chardet                   3.0.4                    pypi_0    pypi
colorama                  0.4.1                    pypi_0    pypi
constantly                15.1.0           py35h28b3542_0  
cycler                    0.10.0                   pypi_0    pypi
decorator                 4.4


UnavailableInvalidChannel: The channel is not accessible or is invalid.
  channel name: lyken/simple
  channel url: https://pypi.anaconda.org/lyken/simple
  error code: 404

You will need to adjust your conda configuration to proceed.
Use `conda config --show channels` to view your configuration's current state,
and use `conda config --show-sources` to view config file locations.




ImportError: dlopen(/Users/Osborn/anaconda3/envs/myTensorflow/lib/python3.5/site-packages/torch/_C.cpython-35m-darwin.so, 9): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/Osborn/anaconda3/envs/myTensorflow/lib/python3.5/site-packages/torch/lib/libshm.dylib
  Reason: image not found